# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,swidnik,51.2190,22.6962,22.61,55,20,4.12,PL,1720085790
1,1,utrik,11.2278,169.8474,27.91,81,76,6.95,MH,1720085793
2,2,north battleford,52.7834,-108.2847,15.78,100,0,3.09,CA,1720085794
3,3,bredasdorp,-34.5322,20.0403,15.62,59,4,4.77,ZA,1720085795
4,4,sorong,-0.8833,131.2500,26.95,79,100,1.04,ID,1720085764


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

map_plot_city = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City"
)


# Display the map
# YOUR CODE HERE
map_plot_city

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE

## Changing the cloudiness condition, as dataset is returning 0 hotel for given condition

ideal_temp_cond_city = ((city_data_df["Max Temp"] > 19) & (city_data_df["Max Temp"] < 28))

#ideal_wind_cond_city = (city_data_df["Wind Speed"] < 4.5)
ideal_wind_cond_city = (city_data_df["Wind Speed"] < 30)

## Feel free to adjust your specifications, but make sure to set a reasonable limit to the number of rows returned by your API requests.
## Changing the cloudiness condition, as dataset is returning 0 hotel for given condition
## ideal_cloud_cond_city = (city_data_df["Cloudiness"] == 0)

ideal_cloud_cond_city = (city_data_df["Cloudiness"] < 10)


city_data_ideal_cond = city_data_df.loc[(ideal_temp_cond_city & ideal_wind_cond_city & ideal_cloud_cond_city ), :]

# Drop any rows with null values
# YOUR CODE HERE

# Drop all rows with missing information
city_data_ideal_cond = city_data_ideal_cond.dropna()

# Display sample data
# YOUR CODE HERE
city_data_ideal_cond

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
6,6,raduzhny,62.0961,77.4750,25.80,35,6,1.40,RU,1720085799
17,17,san luis de la loma,17.2706,-100.8939,24.41,88,8,1.69,MX,1720085813
19,19,uturoa,-16.7333,-151.4333,26.37,76,8,10.05,PF,1720085815
59,59,howard springs,-12.4970,131.0470,26.95,32,0,0.71,AU,1720085874
61,61,camalu,30.8500,-116.0667,19.25,86,7,1.51,MX,1720085877
65,65,stephenville,32.2207,-98.2023,26.80,80,0,3.09,US,1720085882
92,92,rio branco,-9.9747,-67.8100,20.34,94,0,0.00,BR,1720085917
106,106,santa ana de yacuma,-13.7500,-65.4333,21.02,66,6,1.39,BO,1720085936
113,113,manistee,44.2445,-86.3242,20.03,96,0,2.06,US,1720085946
118,118,carnarvon,-24.8667,113.6333,19.98,36,0,6.49,AU,1720085903


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
hotel_df = city_data_ideal_cond.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
hotel_df["Hotel Name"] = ""

# Display sample data
# YOUR CODE HERE
hotel_df = hotel_df[["City","Country","Lat","Lng","Humidity", "Hotel Name"]]
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
6,raduzhny,RU,62.0961,77.4750,35,
17,san luis de la loma,MX,17.2706,-100.8939,88,
19,uturoa,PF,-16.7333,-151.4333,76,
59,howard springs,AU,-12.4970,131.0470,32,
61,camalu,MX,30.8500,-116.0667,86,
65,stephenville,US,32.2207,-98.2023,80,
92,rio branco,BR,-9.9747,-67.8100,94,
106,santa ana de yacuma,BO,-13.7500,-65.4333,66,
113,manistee,US,44.2445,-86.3242,96,
118,carnarvon,AU,-24.8667,113.6333,36,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000 # YOUR CODE HERE
limit = 50
params = {
    # YOUR CODE HERE
    "apiKey":geoapify_key,
    # "limit":limit,
    "format":"json"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}" # YOUR CODE HERE
    params["bias"] = f"proximity:{longitude},{latitude}" # YOUR CODE HERE

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params) # YOUR CODE HERE

    # Convert the API response to JSON format
    name_address = name_address.json()# YOUR CODE HERE

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
raduzhny - nearest hotel: No hotel found
san luis de la loma - nearest hotel: No hotel found
uturoa - nearest hotel: No hotel found
howard springs - nearest hotel: No hotel found
camalu - nearest hotel: No hotel found
stephenville - nearest hotel: No hotel found
rio branco - nearest hotel: No hotel found
santa ana de yacuma - nearest hotel: No hotel found
manistee - nearest hotel: No hotel found
carnarvon - nearest hotel: No hotel found
newman - nearest hotel: No hotel found
mananjary - nearest hotel: No hotel found
camp swift - nearest hotel: No hotel found
aigues-mortes - nearest hotel: No hotel found
zaltan - nearest hotel: No hotel found
laguna - nearest hotel: No hotel found
richards bay - nearest hotel: No hotel found
karratha - nearest hotel: No hotel found
douglas - nearest hotel: No hotel found
tateyama - nearest hotel: No hotel found
salida - nearest hotel: No hotel found
menongue - nearest hotel: No hotel found
tsiombe - nearest hotel: No hotel found
fe

,City,Country,Lat,Lng,Humidity,Hotel Name
6,raduzhny,RU,62.0961,77.4750,35,No hotel found
17,san luis de la loma,MX,17.2706,-100.8939,88,No hotel found
19,uturoa,PF,-16.7333,-151.4333,76,No hotel found
59,howard springs,AU,-12.4970,131.0470,32,No hotel found
61,camalu,MX,30.8500,-116.0667,86,No hotel found
65,stephenville,US,32.2207,-98.2023,80,No hotel found
92,rio branco,BR,-9.9747,-67.8100,94,No hotel found
106,santa ana de yacuma,BO,-13.7500,-65.4333,66,No hotel found
113,manistee,US,44.2445,-86.3242,96,No hotel found
118,carnarvon,AU,-24.8667,113.6333,36,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

map_plot_city_hotel = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)


# Display the map
# YOUR CODE HERE
map_plot_city_hotel

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country)